In [21]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [38]:
# Parameters
device = "cuda" if torch.cuda.is_available() else "cpu"
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250         

In [39]:
with open('data/wizard_in_oz.txt', 'r',  encoding="utf-8") as f:
    text = f.read()

# List of unique characters in the text.
chars = sorted(set(text))
vocab_size = len(chars)

In [40]:
# Tokenizer for charecters.

string_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_string = dict((i, c) for i, c in enumerate(chars))

# Functions for tokenizing the text.
def encode(s: str) -> list:
    return [string_to_int[c] for c in s]

def decode(ints) -> list:
    return ''.join([int_to_string[i] for i in ints])

In [41]:
# Testing functions for tokenizing the text.
helloworld_encoded = encode("hello world")
helloworld_decoded = decode(helloworld_encoded)

print('helloworld_encoded: ' , helloworld_encoded)
print('helloworld_decoded: ', helloworld_decoded)

helloworld_encoded:  [61, 58, 65, 65, 68, 1, 76, 68, 71, 65, 57]
helloworld_decoded:  hello world


In [42]:
# Convert the entire text to the tensor.
data = torch.LongTensor(encode(text))

# Split the data into training and validation sets.
n_splits = int(len(text) * 0.8)
train_data = data[:n_splits]
val_data = data[n_splits:]

In [43]:
block_size = 8

x = train_data[:]

In [44]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

In [45]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



Cet]zXF_-p"e"BsGXtm_nO7iEu]PT.v[YQG
Xn:M.is;;1GOJ"gsc1,.9]lBcV WuXlAg4w﻿F.I'D?AaD*Brk:MXAqn:H
jb﻿FQH*)XtP-dG0Lbrs0ngt-CID,Gp-vQE8
*dBOcyHcPYS﻿sU?r&"Ps)PO﻿IQ9?IoY_JP&LM!dEU?MydO4lnzhy)L,eBt11Vg4R5J!HHBjiwa"e-?8
0nw(kp7!&X?_q.)])H!;A9:,,kci-2 NLs!S6(j& DhQ7Cau1;:1(PlZbU!A:1cC'g16Suet*vV
Ru!Oo'yW*2c W?X4;:hIq[2_j& C1vqr-VgOSXaDcTCliG.3?!O9t*:V;3*n8c1WZD7H[.Vk9 fmIWTIStr[l_j)TC1TOb*Gk''VYS),QddrHccrtT!S﻿IiN6q-ull0Ft"Ag3PtHZnb5i-PU1M zvf)TCQ
We]7ixlZ;QYn0F*AbzP."rq3_-d3pP6!yh8﻿n3o7iUOYw.[r824A&wvMg4


In [46]:
def get_batch(split: str):
    """
    Get a batch of data from the specified split.

    Args:
        split (str): The split to retrieve data from. Can be either 'train' or 'val'.

    Returns:
        A tuple containing the input data (of shape (batch_size, block_size)) and the target data (of shape (batch_size, block_size))
    """
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    """
    Estimate the loss on the training and validation sets.

    Returns:
        A dictionary containing the mean loss on the training and validation sets.
    """
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [47]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.922, val loss: 4.911
step: 250, train loss: 4.864, val loss: 4.825
step: 500, train loss: 4.769, val loss: 4.773
step: 750, train loss: 4.710, val loss: 4.717
4.686432361602783
